In [1]:
import numpy as np

In [2]:
def read_text_file(fname):
    f = open(fname, 'r')
    fl = f.readlines()
    f.close()
    data = []
    
    for line in fl:
        if "#" not in line and "@" not in line:
            data.append(line.split())
    data = np.array(data, dtype='float')
    return data

nc5_raw = [read_text_file("PMFs/NP610c-POPC5-46_PMF.xvg") for i in range(1)]
nc6_raw = [read_text_file("PMFs/NP610c-POPC6-46_PMF_std.xvg") for i in range(1)]

In [3]:
def homogenize_profiles_with_std(raw_data, nice_x=np.linspace(0,5,400)):
    clean_pmf = np.zeros((len(nice_x), len(raw_data)))
    clean_std = np.zeros_like(clean_pmf)
    for i, x in enumerate(nice_x):
        for p, prof in enumerate(raw_data):
            close_ndx = np.argsort(np.abs(prof[:,0]-x))[:2]
            clean_pmf[i,p] = np.mean(prof[close_ndx,1])
            clean_std[i,p] = np.mean(prof[close_ndx,2])
    clean_pmf = clean_pmf - clean_pmf[-1,:]
    
    fitted_pmf = np.zeros_like(clean_pmf)
    fitted_std = np.zeros_like(clean_std)
    for p, (pmf, std) in enumerate(zip(clean_pmf.T, clean_std.T)):
        pol = np.poly1d(np.polyfit(nice_x, pmf, deg=14))
        fitted_pmf[:,p] = pol(nice_x)
        pol_std = np.poly1d(np.polyfit(nice_x, std, deg=14))
        fitted_std[:,p] = pol_std(nice_x)
    fitted_pmf = fitted_pmf - fitted_pmf[-1,:]
    return fitted_pmf, fitted_std

def homogenize_profiles(raw_data, nice_x=np.linspace(0,5,400)):
    clean_pmf = np.zeros((len(nice_x), len(raw_data)))
    for i, x in enumerate(nice_x):
        for p, prof in enumerate(raw_data):
            close_ndx = np.argsort(np.abs(prof[:,0]-x))[:2]
            clean_pmf[i,p] = np.mean(prof[close_ndx,1])
    clean_pmf = clean_pmf - clean_pmf[-1,:]
    
    fitted_pmf = np.zeros_like(clean_pmf)
    for p, pmf in enumerate(clean_pmf.T):
        pol = np.poly1d(np.polyfit(nice_x, pmf, deg=14))
        fitted_pmf[:,p] = pol(nice_x)
    fitted_pmf = fitted_pmf - fitted_pmf[-1,:]
    return fitted_pmf

In [4]:
nice_x = np.linspace(0,5,400) #should have twice as many points since the smoothening will average two points
nc5 = homogenize_profiles(nc5_raw, nice_x=nice_x)
nc6 = homogenize_profiles_with_std(nc6_raw, nice_x=nice_x)

In [5]:
f = open("PMFs/NP610c-POPC5-46_PMF_std.sfu", "w")
for i, x in enumerate(nice_x):
    f.write("{:<10.3f} {:<10.3f} {:<10.3f}\n".format(x, nc5[i,0], nc6[1][i,0]))
f.close()

In [6]:
g6r3_raw = [read_text_file("PMFs/gH-POPC6-46-r{}_PMF.xvg".format(i)) for i in range(3,4)]
g6r4_raw = [read_text_file("PMFs/gH-POPC6-46-r{}_PMF_std.xvg".format(i)) for i in range(4,5)]

In [7]:
nice_x = np.linspace(0,5,400) #should have twice as many points since the smoothening will average two points
g6r3 = homogenize_profiles(g6r3_raw, nice_x=nice_x)
g6r4 = homogenize_profiles_with_std(g6r4_raw, nice_x=nice_x)

In [8]:
f = open("PMFs/gH-POPC6-46-r3_PMF_std.sfu", "w")
for i, x in enumerate(nice_x):
    f.write("{:<10.3f} {:<10.3f} {:<10.3f}\n".format(x, g6r3[i,0], g6r4[1][i,0]))
f.close()